In [8]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder.appName('test-pyspark').getOrCreate()
sqlContext = SQLContext(spark)

from pyspark.sql import functions as F

spark

In [17]:
import os

WEATHER_DATA_PATH = "./data/2019/"

weather_data_files_name = os.listdir(WEATHER_DATA_PATH)

In [18]:
weather_data_files_name

['part-00002-890686c0-c142-4c69-a744-dfdc9eca7df4-c000.csv',
 'part-00001-890686c0-c142-4c69-a744-dfdc9eca7df4-c000.csv',
 'part-00003-890686c0-c142-4c69-a744-dfdc9eca7df4-c000.csv',
 'part-00000-890686c0-c142-4c69-a744-dfdc9eca7df4-c000.csv',
 'part-00004-890686c0-c142-4c69-a744-dfdc9eca7df4-c000.csv']

In [31]:
wetaher_data_files_df = []

for file_name in weather_data_files_name:
    file_path = WEATHER_DATA_PATH + file_name
    df = spark.read.csv(WEATHER_DATA_PATH + weather_data_files_name[1],
                        sep=",",
                        inferSchema="true",
                        header="true")
    wetaher_data_files_df.append(df)

wetaher_data_files_df[0].show(2)

+------+-----+--------+----+----+------+-----+-----+----+-----+-----+----+----+-----+-----+------+
|STN---| WBAN|YEARMODA|TEMP|DEWP|   SLP|  STP|VISIB|WDSP|MXSPD| GUST| MAX| MIN| PRCP| SNDP|FRSHTT|
+------+-----+--------+----+----+------+-----+-----+----+-----+-----+----+----+-----+-----+------+
|724233|53841|20190101|49.2|41.6|1020.6|989.3| 10.0| 6.4| 18.1| 28.9|66.0|39.0|1.25G|999.9|     0|
|724233|53841|20190102|42.6|38.1|1022.3|993.1|  9.3| 2.8|  8.0|999.9|55.0|41.0|0.00G|999.9|     0|
+------+-----+--------+----+----+------+-----+-----+----+-----+-----+----+----+-----+-----+------+
only showing top 2 rows



In [35]:
for file in wetaher_data_files_df:
    print(df.count())

894455
894455
894455
894455
894455


In [36]:
import functools 

def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

weather_data_df = unionAll(wetaher_data_files_df)

In [38]:
weather_data_df.show(2)

+------+-----+--------+----+----+------+-----+-----+----+-----+-----+----+----+-----+-----+------+
|STN---| WBAN|YEARMODA|TEMP|DEWP|   SLP|  STP|VISIB|WDSP|MXSPD| GUST| MAX| MIN| PRCP| SNDP|FRSHTT|
+------+-----+--------+----+----+------+-----+-----+----+-----+-----+----+----+-----+-----+------+
|724233|53841|20190101|49.2|41.6|1020.6|989.3| 10.0| 6.4| 18.1| 28.9|66.0|39.0|1.25G|999.9|     0|
|724233|53841|20190102|42.6|38.1|1022.3|993.1|  9.3| 2.8|  8.0|999.9|55.0|41.0|0.00G|999.9|     0|
+------+-----+--------+----+----+------+-----+-----+----+-----+-----+----+----+-----+-----+------+
only showing top 2 rows



In [37]:
weather_data_df.count()

4472275

In [43]:
weather_data_df.printSchema()

root
 |-- STN---: integer (nullable = true)
 |-- WBAN: integer (nullable = true)
 |-- YEARMODA: integer (nullable = true)
 |-- TEMP: double (nullable = true)
 |-- DEWP: double (nullable = true)
 |-- SLP: double (nullable = true)
 |-- STP: double (nullable = true)
 |-- VISIB: double (nullable = true)
 |-- WDSP: double (nullable = true)
 |-- MXSPD: double (nullable = true)
 |-- GUST: double (nullable = true)
 |-- MAX: string (nullable = true)
 |-- MIN: string (nullable = true)
 |-- PRCP: string (nullable = true)
 |-- SNDP: double (nullable = true)
 |-- FRSHTT: integer (nullable = true)



In [60]:
weather_data_df = weather_data_df.withColumnRenamed("STN---", "STN_NO")
weather_data_df.printSchema()

root
 |-- STN_NO: integer (nullable = true)
 |-- WBAN: integer (nullable = true)
 |-- YEARMODA: integer (nullable = true)
 |-- TEMP: double (nullable = true)
 |-- DEWP: double (nullable = true)
 |-- SLP: double (nullable = true)
 |-- STP: double (nullable = true)
 |-- VISIB: double (nullable = true)
 |-- WDSP: double (nullable = true)
 |-- MXSPD: double (nullable = true)
 |-- GUST: double (nullable = true)
 |-- MAX: string (nullable = true)
 |-- MIN: string (nullable = true)
 |-- PRCP: string (nullable = true)
 |-- SNDP: double (nullable = true)
 |-- FRSHTT: integer (nullable = true)



In [ ]:
# 2. Join the stationlist.csv with the countrylist.csv to get the full country name
# for each station number.

In [66]:
staion_list_df = spark.read.csv("./stationlist.csv",
                                sep=",",
                                inferSchema="true",
                                header="true")

staion_list_df.show(2)

+------+------------+
|STN_NO|COUNTRY_ABBR|
+------+------------+
|012240|          NO|
|020690|          SW|
+------+------------+
only showing top 2 rows



In [67]:
country_list_df = spark.read.csv("./countrylist.csv",
                                sep=",",
                                inferSchema="true",
                                header="true")

country_list_df.show(2)

+------------+-------------------+
|COUNTRY_ABBR|       COUNTRY_FULL|
+------------+-------------------+
|          AA|              ARUBA|
|          AC|ANTIGUA AND BARBUDA|
+------------+-------------------+
only showing top 2 rows



In [68]:
full_country_startion_list_df = country_list_df.join(staion_list_df, ['COUNTRY_ABBR'])
full_country_startion_list_df.show(3)

+------------+------------+------+
|COUNTRY_ABBR|COUNTRY_FULL|STN_NO|
+------------+------------+------+
|          NO|      NORWAY|012240|
|          SW|      SWEDEN|020690|
|          SW|      SWEDEN|020870|
+------------+------------+------+
only showing top 3 rows



In [69]:
# 3. Join the global weather data with the full country names by station number.

In [70]:
weather_data_country_name_df = weather_data_df.join(full_country_startion_list_df, ["STN_NO"])
weather_data_country_name_df.show(2)

+------+-----+--------+----+----+------+------+-----+----+-----+-----+-----+-----+-----+-----+------+------------+------------+
|STN_NO| WBAN|YEARMODA|TEMP|DEWP|   SLP|   STP|VISIB|WDSP|MXSPD| GUST|  MAX|  MIN| PRCP| SNDP|FRSHTT|COUNTRY_ABBR|COUNTRY_FULL|
+------+-----+--------+----+----+------+------+-----+----+-----+-----+-----+-----+-----+-----+------+------------+------------+
| 64590|99999|20190101|43.0|40.9|1033.1|1013.1|999.9| 9.6| 11.7|999.9|45.1*|38.7*|0.04F|999.9|     0|          BE|     BELGIUM|
| 64590|99999|20190102|38.6|33.9|1038.7|1018.4|999.9| 7.0|  9.7|999.9| 41.0| 35.6|0.00E|999.9|     0|          BE|     BELGIUM|
+------+-----+--------+----+----+------+------+-----+----+-----+-----+-----+-----+-----+-----+------+------------+------------+
only showing top 2 rows



In [ ]:
# Step 2

# 1. Which country had the hottest average mean temperature over the year?

In [121]:
# The "TEMP" column NA values are shows with 9999.9. We first replace these values by nan
weather_data_country_name_df = weather_data_country_name_df.replace(9999.9, None, subset=["TEMP"])

In [120]:
weather_data_country_name_df.describe("TEMP").show()

+-------+------------------+
|summary|              TEMP|
+-------+------------------+
|  count|           4433950|
|   mean|52.586750076117205|
| stddev|22.692241474316322|
|    min|             -67.9|
|    max|             109.9|
+-------+------------------+



In [ ]:
# From the above there is no values with 9999.9 in the dataframe
# Since we are calculating the mean temprture here, we do not need to replace the values
# however, if further processing is required, we could replace them with mean or median of TEMP values for each country.

In [79]:
# First compute the averaage temprature for day over all statiions in each country

country_day_mean_temp_df = weather_data_country_name_df.groupBy(["COUNTRY_FULL", 'YEARMODA'])\
                                                               .agg(F.mean("TEMP").alias("average_day_mean_temp"))\
                                                               .orderBy(F.desc('average_day_mean_temp'))

country_day_mean_temp_df.show(5)

+------------+--------+---------------------+
|COUNTRY_FULL|YEARMODA|average_day_mean_temp|
+------------+--------+---------------------+
|      KUWAIT|20190707|                108.1|
|      KUWAIT|20190725|                107.2|
|      KUWAIT|20190609|                107.0|
|      KUWAIT|20190627|                107.0|
|      KUWAIT|20190628|                106.9|
+------------+--------+---------------------+
only showing top 5 rows



In [106]:
# Now caluclate mean temprature over all days for each country

country_mean_temp_df = country_day_mean_temp_df.groupBy(["COUNTRY_FULL"])\
                                               .agg(F.mean("average_day_mean_temp").alias("average_mean_temp"))\
                                               .orderBy(F.desc('average_mean_temp'))

country_mean_temp_df.show(5)

+------------+-----------------+
|COUNTRY_FULL|average_mean_temp|
+------------+-----------------+
|       NIGER|86.85552511415523|
|        OMAN|83.77134986225896|
|       QATAR|83.58379120879125|
|    THAILAND|83.00479452054793|
|    COLOMBIA|82.89273972602739|
+------------+-----------------+
only showing top 5 rows



In [ ]:
# From the above results, NIGER was the hottest country based on average yearly temrature

In [ ]:
# 2. Which country had the most consecutive days of tornadoes/funnel cloud
# formations?

In [138]:
from pyspark.sql.types import IntegerType

weather_data_country_name_tornado_df = weather_data_country_name_df.withColumn("To_Fu_Cl",
                                                                               F.col("FRSHTT").cast(IntegerType()) % 10)
weather_data_country_name_tornado_df.show(5)

+------+-----+--------+----+----+------+------+-----+----+-----+-----+-----+-----+-----+-----+------+------------+------------+--------+
|STN_NO| WBAN|YEARMODA|TEMP|DEWP|   SLP|   STP|VISIB|WDSP|MXSPD| GUST|  MAX|  MIN| PRCP| SNDP|FRSHTT|COUNTRY_ABBR|COUNTRY_FULL|To_Fu_Cl|
+------+-----+--------+----+----+------+------+-----+----+-----+-----+-----+-----+-----+-----+------+------------+------------+--------+
| 64590|99999|20190101|43.0|40.9|1033.1|1013.1|999.9| 9.6| 11.7|999.9|45.1*|38.7*|0.04F|999.9|     0|          BE|     BELGIUM|       0|
| 64590|99999|20190102|38.6|33.9|1038.7|1018.4|999.9| 7.0|  9.7|999.9| 41.0| 35.6|0.00E|999.9|     0|          BE|     BELGIUM|       0|
| 64590|99999|20190103|37.2|35.4|1040.9|1020.5|999.9| 5.2|  9.7|999.9| 40.6|34.3*|0.01E|999.9|     0|          BE|     BELGIUM|       0|
| 64590|99999|20190104|35.7|34.3|1039.6|1019.2|999.9| 6.7| 11.7|999.9| 38.7| 32.5|0.01F|999.9|     0|          BE|     BELGIUM|       0|
| 64590|99999|20190105|39.7|39.5|1035.4|1

In [149]:
#  Create new column that shows wheather there was any Torando in each country in each day in all stations

grouped_weather_data_country_name_tornado_df = weather_data_country_name_tornado_df.groupBy(["COUNTRY_FULL", 'YEARMODA'])\
                                                                           .agg(F.max('To_Fu_Cl').alias("Any_To_Fu_Cl"))

grouped_weather_data_country_name_tornado_df.show(5)

+------------+--------+------------+
|COUNTRY_FULL|YEARMODA|Any_To_Fu_Cl|
+------------+--------+------------+
|     BELGIUM|20190215|           0|
| SWITZERLAND|20190105|           0|
| SWITZERLAND|20190307|           0|
| SWITZERLAND|20190309|           0|
| SWITZERLAND|20190918|           0|
+------------+--------+------------+
only showing top 5 rows



In [159]:
# Create a column to show if there was any torndao in the previous day

from pyspark.sql.window import Window
windowSpec = Window().partitionBy(['COUNTRY_FULL']).orderBy('YEARMODA')
grouped_weather_data_country_name_tornado_df1 = grouped_weather_data_country_name_tornado_df.withColumn("Any_To_Fu_Cl_prev_day", F.lag("Any_To_Fu_Cl", 1).over(windowSpec))


# Creete a column to show if there was any tonado in current day and previous day

grouped_weather_data_country_name_tornado_df2 = grouped_weather_data_country_name_tornado_df1.withColumn("Consecutive_tonado",
                                                                                                       (F.col("Any_To_Fu_Cl")==1) & (F.col('Any_To_Fu_Cl_prev_day')==1)
                                                                                                      )

# For each country calculate number of 
grouped_weather_data_country_name_tornado_df2.groupby(['COUNTRY_FULL']).agg(F.sum("Consecutive_tonado")).sort('Consecutive_tonado')


AnalysisException: cannot resolve 'sum(`Consecutive_tonado`)' due to data type mismatch: function sum requires numeric types, not boolean;;
'Aggregate [COUNTRY_FULL#3012], [COUNTRY_FULL#3012, sum(Consecutive_tonado#7873) AS sum(Consecutive_tonado)#7885]
+- Project [COUNTRY_FULL#3012, YEARMODA#1029, Any_To_Fu_Cl#7736, Any_To_Fu_Cl_prev_day#7868, ((Any_To_Fu_Cl#7736 = 1) AND (Any_To_Fu_Cl_prev_day#7868 = 1)) AS Consecutive_tonado#7873]
   +- Project [COUNTRY_FULL#3012, YEARMODA#1029, Any_To_Fu_Cl#7736, Any_To_Fu_Cl_prev_day#7868]
      +- Project [COUNTRY_FULL#3012, YEARMODA#1029, Any_To_Fu_Cl#7736, Any_To_Fu_Cl_prev_day#7868, Any_To_Fu_Cl_prev_day#7868]
         +- Window [lag(Any_To_Fu_Cl#7736, 1, null) windowspecdefinition(COUNTRY_FULL#3012, YEARMODA#1029 ASC NULLS FIRST, specifiedwindowframe(RowFrame, -1, -1)) AS Any_To_Fu_Cl_prev_day#7868], [COUNTRY_FULL#3012], [YEARMODA#1029 ASC NULLS FIRST]
            +- Project [COUNTRY_FULL#3012, YEARMODA#1029, Any_To_Fu_Cl#7736]
              

In [ ]:
# There is a small error in the above code that needs to be finxed, due to time limit I will not fx it

In [ ]:
# 3. Which country had the second highest average mean wind speed over the year?

In [128]:
weather_data_country_name_df.describe('WDSP').show()

+-------+------------------+
|summary|              WDSP|
+-------+------------------+
|  count|           4433950|
|   mean| 28.10708938982154|
| stddev|146.24980820722186|
|    min|               0.0|
|    max|             999.9|
+-------+------------------+



In [129]:
# The "WDSPD" column NA values are shows with 999.9. We first replace these values by None
weather_data_country_name_df = weather_data_country_name_df.replace(999.9, None, subset=["WDSP"])

In [130]:
weather_data_country_name_df.describe('WDSP').show()

+-------+-----------------+
|summary|             WDSP|
+-------+-----------------+
|  count|          4335825|
|   mean|6.114232354857499|
| stddev|4.107605511186825|
|    min|              0.0|
|    max|             71.8|
+-------+-----------------+



In [ ]:
# From the above there is no values with 999.9 in the dataframe anymore
# Since we are calculating the mean windspeed here, we do not need to replace the values
# however, if further processing is required, we could replace them with mean or median of WDSP values for each country.

In [131]:
# First compute the averaage wind speed for day over all statiions in each country

country_day_mean_wind_spd_df = weather_data_country_name_df.groupBy(["COUNTRY_FULL", 'YEARMODA'])\
                                                               .agg(F.mean("WDSP").alias("average_day_wind_spd"))

country_day_mean_wind_spd_df.show(5)

+------------+--------+--------------------+
|COUNTRY_FULL|YEARMODA|average_day_wind_spd|
+------------+--------+--------------------+
|     BELGIUM|20190215|   5.466666666666667|
| SWITZERLAND|20190105|   6.677083333333333|
| SWITZERLAND|20190307|   9.391666666666667|
| SWITZERLAND|20190309|   9.470833333333333|
| SWITZERLAND|20190918|   5.672916666666667|
+------------+--------+--------------------+
only showing top 5 rows



In [132]:
# Now caluclate mean wind speed over all days for each country

country_mean_wind_spd_df = country_day_mean_wind_spd_df.groupBy(["COUNTRY_FULL"])\
                                               .agg(F.mean("average_day_wind_spd").alias("average_wind_spd"))\
                                               .orderBy(F.desc('average_wind_spd'))

country_mean_wind_spd_df.show(5)

+--------------------+------------------+
|        COUNTRY_FULL|  average_wind_spd|
+--------------------+------------------+
|FRENCH SOUTHERN A...|18.989589041095886|
|           ST. LUCIA|15.469672131147542|
|FALKLAND ISLANDS ...| 15.34630136986301|
|           MAURITIUS|12.767671232876712|
|          ANTARCTICA|  12.0260682776128|
+--------------------+------------------+
only showing top 5 rows



In [ ]:
#  From the above results, ST. LUCIA has the second highest wind speed over the year